In [1]:
import numpy as np
import cv2
import os
import logging
from twilio.rest import Client
from datetime import datetime

In [2]:
# Define face cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Creating a function to detect faces
def detect_face(img_array_gray):
    faces = face_cascade.detectMultiScale(img_array_gray,scaleFactor=1.1, minNeighbors=5)
    for (W, H, w, h) in faces:
        face_array = img_array_gray[H:H+h, W:W+w]
        eyes = eye_cascade.detectMultiScale(face_array)
        if len(eyes) >= 2:   # Ensure at least two eyes are detected
            return face_array, (W, H, w, h)
    return None, None


# Creating a function to draw rectangle
def draw_rectangle(frame, W, H, w, h, thickness=2):
    cv2.rectangle(frame, (W, H), (W+w, H+h), (255, 255, 255), thickness)
    return frame

# Create a function to put text on the detected image
def put_text(frame, text, W, H, col=(0,255,0), scale=2):
    cv2.putText(frame, text, (W, H), cv2.FONT_HERSHEY_COMPLEX, scale, col, 2)
    return frame

main_path = r'C:\Users\RONO\Desktop\Open_cv_final\data'
names = os.listdir(main_path)
train_img = []
label = []
rectangle = []

# Create a dictionary to map label names to integers
label_dict = {name: idx for idx, name in enumerate(names)}

for name in names:
    name_path = os.path.join(main_path, name)
    img_names = os.listdir(name_path)
    for img in img_names:
        img_path = os.path.join(name_path, img)
        gray = cv2.imread(img_path, 0)
        face_array, position = detect_face(gray)
        if face_array is not None:
            train_img.append(face_array)
            label.append(label_dict[name])  # Append integer label
            rectangle.append(position)

# Convert labels to numpy array of type int32
labels = np.array(label, dtype=np.int32)

# Initializing Recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Train recognizer
recognizer.train(train_img, labels)

# Save the trained model
recognizer.save('face_recognizer_model.yml')

print("Training completed and model saved.")


Training completed and model saved.


In [3]:
def predict_array_image(gray_img):
    frame, rect = detect_face(gray_img)
    if frame is not None:
        label, conf = recognizer.predict(frame)
        return label, conf, rect
    else:
        return None, None, None

In [4]:
# Creating a function to send alert sms

def send_sms_alert():
    account_sid = 'enter your account sid'
    auth_token = 'enter your auth token'
    client = Client(account_sid, auth_token)

    message = client.messages.create(
        body="Security Alert: Unauthorized Access Detected",
        from_='enter your twilio number', 
        to='enter your phone number' 
    )
    logging.info(f"SMS sent successfully: {message.sid}")

In [7]:
recognizer.read('face_recognizer_model.yml')

cap = cv2.VideoCapture(0)
while True:
    ret,frame = cap.read()
    gray_img = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    label,conf,rect = predict_array_image(gray_img)
    if rect is not None:
        c=0
        W,H,w,h = rect
        if conf < 55:
            name = names[label]
            draw_rectangle(frame,W,H,w,h)
            put_text(frame, name, W, H)
        else:
            put_text(frame, "Unknown", W, H, col=(0, 0, 255))
            draw_rectangle(frame,W,H,w,h)
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            image_path = f"unknown_{timestamp}.jpg"
            cv2.imwrite(image_path, frame)
            send_sms_alert()                   # Send SMS alert for unauthorized access
            logging.info(f"Unauthorized access detected at {timestamp}")
            
        while c<=20:
            c+=1
    cv2.imshow('frame',frame)
    if cv2.waitKey(100) == ord('k'):
        break
cv2.destroyAllWindows()
cap.release()